In [1]:
import pickle

In [2]:
import glob
import errno
import unicodedata

In [ ]:
testing = "1928_in_association_football 0 The following are the football -LRB- soccer -RRB- events of the year 1928 throughout the world ."
testing1 = testing.split()
print(testing1)
testing2 = " ".join(testing1)
print(testing2)

# Writing complete wiki file to pickle

In [ ]:
path = 'wiki-pages-text/*'
files = glob.glob(path)
count = 0
faultySentenceIdCount = 0
wikiShards = {}

for name in files:
    count += 1
    with open(name, encoding="utf8") as f:
        for line in f:
            eachLine = line.split()
            docId = unicodedata.normalize('NFD', eachLine[0])
            tempSentenceId = eachLine[1]
            if tempSentenceId.isdigit():
                sentenceId = int(tempSentenceId)
                sentence = " ".join(eachLine[2:])
                newSentence = {sentenceId:sentence}
                
                if docId in wikiShards:
                    wikiShards[docId].append(newSentence)
                    
                else:
                    wikiShards[docId] = []
                    wikiShards[docId].append(newSentence)
            else:
                faultySentenceIdCount += 1
print("Total files : " + count)
print("Sentence without ID : " + faultySentenceIdCount)

In [ ]:
with open('allWikiShardsDict.pickle', 'wb') as handle:
    pickle.dump(wikiShards, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [7]:
with open('allWikiShardsDict.pickle', 'rb') as handle:
    allWikiDocs = pickle.load(handle)
# print (wikiShards == allWikiDocs)
print("done")

done


In [ ]:
print(len(allWikiDocs.keys()))

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


sw = stopwords.words('english') + ['!',',','.','?','\s',"\n","-lrb-","-rrb-"]
def pre_process(sentence):
    tokens = nltk.word_tokenize(sentence)
    tokens = [lemmatize(t) for t in tokens]
    tokens = [t.lower() for t in tokens]
    tokens =  [t for t in tokens if t not in sw]
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
print(pre_process("Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League."))

['colin', 'kaepernick', 'become', 'start', 'quarterback', '49ers', '63rd', 'season', 'national', 'football', 'league']


In [6]:
import json
all_claims_keywords = set()
with open('devset.json', 'r') as f:
    info = json.load(f)

for key in info:
    all_claims_keywords.update(pre_process(info[key]["claim"]))

print(len(all_claims_keywords))


with open('train.json', 'r') as f:
    info = json.load(f)

for key in info:
    all_claims_keywords.update(pre_process(info[key]["claim"]))

print(len(all_claims_keywords))

with open('test-unlabelled.json', 'r') as f:
    info = json.load(f)

for key in info:
    all_claims_keywords.update(pre_process(info[key]["claim"]))

print(len(all_claims_keywords))

with open('all_claims_keywords.pickle', 'wb') as handle:
    pickle.dump(all_claims_keywords, handle, protocol=pickle.HIGHEST_PROTOCOL)

4411
21050
22548


In [ ]:
listKeys = list(allWikiDocs.keys())

docId_int_dict = {}

i = 0
for key in listKeys:
    docId_int_dict[key] = i
    i += 1

with open('docIdToIntDict.pickle', 'wb') as handle:
    pickle.dump(docId_int_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


In [ ]:
with open('docIdToIntDict.pickle', 'rb') as handle:
    docIdIntDict = pickle.load(handle)
    
#print(docId_int_dict == docIdIntDict)

In [ ]:
import json

with open('train.json', 'r') as f:
    trainJsonDict = json.load(f)
    
print(len(trainJsonDict.keys()))

In [ ]:
trainingWikiDocs = {}
for key in trainJsonDict.keys():
    tempEvidenceList = trainJsonDict[key]['evidence']  
    for evidence in tempEvidenceList:
        foundDoc = unicodedata.normalize('NFD', evidence[0])
        if (foundDoc in allWikiDocs) and (foundDoc not in trainingWikiDocs):
            trainingWikiDocs[foundDoc] = allWikiDocs[foundDoc]
            
print("done")

In [ ]:
with open('allTrainShards.pickle', 'wb') as handle:
    pickle.dump(trainingWikiDocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('allTrainShards.pickle', 'rb') as handle:
    allTrainDocs = pickle.load(handle)
    
#print(allTrainDocs == trainingWikiDocs)

In [ ]:
count = 0
for key in allTrainDocs.keys():
    print (key," ",allTrainDocs[key])
    count += 1
    print()
    if count > 10:
        break

In [ ]:
sentenceList = allTrainDocs['Nikolaj_Coster-Waldau']
#for sentence in sentenceList:
#    rawSentence = sentence.values()
totalSentence = []

sentence = sentenceList[0]
rawSentence = list(sentence.values())[0]
totalSentence.append(rawSentence)


sentence2 = sentenceList[1]
rawSentence2 = list(sentence2.values())[0]
totalSentence.append(rawSentence2)


#totalSentence = rawSentence+" "+rawSentence2

sentence3 = sentenceList[2]
rawSentence3 = list(sentence3.values())[0]

totalSentence.append(rawSentence3)

finalRawDoc = " ".join(totalSentence)
print(finalRawDoc)

In [ ]:
import gensim

In [ ]:
i = 0
docToIntMap = {}
justDocumentList = []
for doc in allWikiDocs.keys():
    sentenceList = allWikiDocs[doc]
    rawSentenceList = []
    for sentence in sentenceList:
        rawSentence = list(sentence.values())[0]
        rawSentenceList.append(rawSentence)
    finalRawDoc = " ".join(rawSentenceList)
    docToIntMap[doc] = i
    justDocumentList.append(finalRawDoc)   
    i += 1

print(type(justDocumentList))

In [ ]:
with open('justDocumentList.pickle', 'wb') as handle:
    pickle.dump(justDocumentList, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('docToIntMap.pickle', 'wb') as handle:
    pickle.dump(docToIntMap, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('justDocumentList.pickle', 'rb') as handle:
    justDocumentList = pickle.load(handle)

In [ ]:
with open('docToIntMap.pickle', 'rb') as handle:
    docToIntMap = pickle.load(handle)

In [ ]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in justDocumentList]
#print(gen_docs)

In [ ]:
gen_docs = [[w for w in text.split()] 
            for text in justDocumentList]
#print(gen_docs)

In [ ]:
dictionary = gensim.corpora.Dictionary(gen_docs)

In [ ]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [ ]:
with open('corpusTemp.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
tf_idf = gensim.models.TfidfModel(corpus)

In [ ]:
s = 0
for i in corpus:
    s += len(i)
print(s)

In [ ]:
sims = gensim.similarities.Similarity('E:/(4) Semester 1 February 2019/COMP90042 Web Search and Text Analysis/Project Our/',tf_idf[corpus],
                                      num_features=len(dictionary))

In [ ]:
query_doc = [w.lower() for w in word_tokenize("The Ten Commandments is an epic film.")]
#print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
#print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
#print(query_doc_tf_idf)

In [ ]:
print(sims)
print(type(sims))

In [ ]:
print(type(sims[query_doc_tf_idf]))

In [ ]:
import numpy as np
#mylist = [3,2,1,5,0]
ind = np.argmax(sims[query_doc_tf_idf])
print(ind)



In [ ]:
print(list(docToIntMap.keys())[list(docToIntMap.values()).index(ind)])